In [1]:
import os, pandas as pd
# Add asset from file system
churn_df = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/training.csv')

def preprocess(data_file,column_file):
    churn_df = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/{}'.format(data_file))
    columns = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/{}'.format(column_file),header=None)
    column_names = list(columns[0])
    churn_df.columns = column_names
    return churn_df

churn_df = preprocess('training.csv','column_sequence.txt')
churn_df.head()

,last_30_days,last_60_days,last_90_days,no_of_orders,product_amount,no_of_payments,payment_amount,churn_flag
0,0,0,0,3,76.97,0,0.00,1
1,1,2,3,1,5.99,6,35.94,0
2,1,1,1,2,70.98,3,76.97,0
3,0,0,0,1,5.99,3,17.97,0
4,1,2,3,1,5.99,6,35.94,0


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin

class ChurnTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, file_name):
        self.file_name = file_name

    def fit(self, X, y):
        return self

    def transform(self, X,file_name):
        #churn_df = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/{}'.format(X))
        columns = pd.read_csv(os.environ['DSX_PROJECT_DIR']+'/datasets/{}'.format(file_name),header=None)
        column_names = list(columns[0])
        X.columns = column_name
        return X
       


In [3]:
import sklearn
pd.options.display.max_columns = 999
import numpy as np
import urllib3, requests, json
import warnings
warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency,ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.cross_validation import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_curve, roc_auc_score

In [4]:
x = churn_df.loc[:,churn_df.columns!='churn_flag']
y = churn_df['churn_flag']


In [8]:
# split the data to training and testing set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42, stratify=y)

In [9]:
# fit the model
import sklearn.pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

random_forest = RandomForestClassifier()
lr = LogisticRegression()
steps = [('random_forest', random_forest)]
#steps = [('lr', lr)]
pipeline = sklearn.pipeline.Pipeline(steps)
model=pipeline.fit(X_train, y_train)

In [10]:
### call pipeline.predict() on your X_test data to make a set of test predictions
y_prediction = pipeline.predict( X_test )

### test your predictions using sklearn.classification_report()

report = sklearn.metrics.classification_report(y_test, y_prediction )
### and print the report
print(report)

             precision    recall  f1-score   support

          0       0.69      0.91      0.79     10468
          1       0.87      0.60      0.71     10469

avg / total       0.78      0.75      0.75     20937



In [54]:
# View the test data before saving
y_test_df = pd.DataFrame(y_test,columns=['churn_flag'])
y_test_df.head()

,churn_flag
87430,1
3744,0
119565,1
78572,1
16237,0


In [16]:
# Set grid search params
param_range =[1,2]
from sklearn.grid_search import GridSearchCV
grid_params = [{'clf__criterion': ['gini', 'entropy'],
		'clf__min_samples_leaf': param_range,
		'clf__max_depth': param_range,
		'clf__min_samples_split': param_range[1:],
		'clf__presort': [True, False]}]

# Construct grid search
gs = GridSearchCV(estimator=pipeline,param_grid=grid_params,
            scoring='accuracy',
            cv=10)

# Fit using grid search
gs.fit(X_train, y_train)

# Best accuracy
print('Best accuracy: %.3f' % gs.best_score_)

ValueError: Invalid parameter clf for estimator Pipeline(memory=None,
     steps=[('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]). Check the list of available parameters with `estimator.get_params().keys()`.